In [1]:
from collections import defaultdict
import requests
import pandas as pd

### 0. Get list of red links parent titles (in English)

In [2]:
#path to data
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/uk_red_links/'

all_uk_redlinks_with_parent_ids = pd.read_csv(PATH_TO_DATA+'df_allukredlinks_withentitle.csv', encoding = 'UTF-8')

all_uk_redlinks_with_parent_ids = all_uk_redlinks_with_parent_ids.drop_duplicates()

en_name = all_uk_redlinks_with_parent_ids[['ll_title']]

list_of_en_name = en_name['ll_title'].tolist()

len(list_of_en_name)

### 1. Get Wikidata item id with Wikipedia page title

In [11]:
def get_id_from_nested_dict(dict_):
    try:
        key1 = list(dict_)[1]
        query_dict = dict_.get(key1)
        key2 = list(query_dict)[0]
        pages_dict = query_dict.get(key2)
        key3 = list(pages_dict)[0]
        more_dict = pages_dict.get(key3)
        key4 = list(more_dict)[3]
        pageprops_dict = more_dict.get(key4)
        id_ = pageprops_dict.get('wikibase_item')
    except:
        id_ = None
    return id_

In [12]:
# query Wikidata item id with Wikipedia page title
Wikidata_ids = []
i = 0
for title in list_of_en_name:
    response = requests.get("https://en.wikipedia.org/w/api.php?\
                        action=query&\
                        prop=pageprops&\
                        ppprop=wikibase_item&\
                        format=json&\
                        titles="+title)
    dict_ = response.json()
    if i % 10 == 0:
        print(i)
    i += 1
    id_ = get_id_from_nested_dict(dict_)
    Wikidata_ids.append(id_)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

SSLError: ("bad handshake: SysCallError(104, 'ECONNRESET')",)

In [17]:
len(Wikidata_ids)

3245

In [18]:
len(set(Wikidata_ids))

1518

### 2. Get instance_of a Wikidata item

In [79]:
def get_instance_id_from_nested_dict(dict_):
    try:
        key1 = list(dict_)[0]
        entities_dict = dict_.get(key1)
        key2 = list(entities_dict)[0]
        item_dict = entities_dict.get(key2)
        claims_dict = item_dict.get('claims')
        instance_dict = claims_dict.get('P31')
        id_dict = instance_dict[0]
        mainsnak_dict = id_dict.get('mainsnak')
        datavalue_dict = mainsnak_dict.get('datavalue')
        value_dict = datavalue_dict.get('value')
        instance_id = value_dict.get('id')
    except:
        instance_id = None
    return instance_id


In [80]:
# query instance_of a wikidata item
instance_ids = []
i = 0
for wikidata_id in Wikidata_ids:
    try:
        response = requests.get("https://www.wikidata.org/w/api.php?\
                            action=wbgetentities&\
                            props=claims&\
                            format=json&\
                            ids="+wikidata_id+"&languages=en")
        dict_ = response.json()
        if i % 10 == 0:
            print(i)
        i += 1
        instance_id = get_id_from_nested_dict(dict_)
    except:
        instance_id = None
    instance_ids.append(instance_id)

0
10
20
30
40
50
60
70
80
90


TypeError: must be str, not NoneType

### 3. Get the corresponding name of the instance

In [103]:
def get_instance_name_from_nested_dict(dict_):
    try:
        entities_dict = dict_.get('entities')
        key = list(entities_dict)[0]
        id_key = entities_dict.get(key)
        labels_dict = id_key.get('labels')
        en_dict = labels_dict.get('en')
        final_value = en_dict.get('value')
    except:
        final_value = None
    return final_value

In [108]:
# query the corresponding name of the instance
instance_names = []
i = 0
for instance_id in instance_ids:
    try:
        response = requests.get("https://www.wikidata.org/w/api.php?\
                                action=wbgetentities&\
                                props=labels&\
                                format=json&\
                                ids="+instance_id+"&languages=en")
        dict_ = response.json()
        instance_name = get_instance_name_from_nested_dict(dict_)
    except:
        instance_name = None
    if i % 10 == 0:
        print(i)
    i += 1
    instance_names.append(instance_name)

0
10
20
30
40
50
60
70
80
90


In [110]:
instance_names

['academic discipline',
 'academic discipline',
 'Wikimedia disambiguation page',
 'academic discipline',
 None,
 None,
 'Mating system',
 None,
 None,
 None,
 'branch of science',
 'branch of science',
 'branch of science',
 'branch of science',
 'Wikimedia disambiguation page',
 'branch of science',
 'concept',
 'aspect of history',
 None,
 'branch of science',
 'branch of science',
 'branch of science',
 'branch of science',
 'Wikimedia list article',
 'branch of science',
 'Wikimedia list article',
 'branch of science',
 'Wikimedia list article',
 'branch of science',
 'Wikimedia list article',
 'branch of science',
 'taxon',
 'taxon',
 'branch of science',
 'taxon',
 'branch of science',
 None,
 None,
 'extensive physical quantity',
 None,
 None,
 None,
 None,
 'clade',
 None,
 'languoid class',
 'languoid class',
 'languoid class',
 'profession',
 'languoid class',
 'languoid class',
 None,
 'languoid class',
 'languoid class',
 'languoid class',
 None,
 None,
 'animal organ syst

### 4. Transform results to a dictionary

In [112]:
categories_stats = defaultdict(int)
for w in instance_names:
    categories_stats[w] += 1

In [113]:
categories_stats

defaultdict(int,
            {'academic discipline': 3,
             'Wikimedia disambiguation page': 2,
             None: 17,
             'Mating system': 1,
             'branch of science': 15,
             'concept': 1,
             'aspect of history': 1,
             'Wikimedia list article': 4,
             'taxon': 3,
             'extensive physical quantity': 1,
             'clade': 1,
             'languoid class': 10,
             'profession': 1,
             'animal organ system': 1,
             'literary genre': 1,
             'language': 1,
             'planned language': 15,
             'human': 2,
             'letter having circumflex': 6,
             'year': 2,
             'determinator for date of periodic occurrence': 2,
             'history of a country or state': 1,
             'military unit': 1})